In [3]:
# Fixing imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.datasets import fetch_openml
import time

In [4]:
x, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [5]:
# Fixing ys and xs for training
# x = x.reshape(len(x), 28, 28)

real_y = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] for i in range(len(y))])
for i in range(len(y)):
    real_y[i, int(y[i])] = 1.0

In [6]:
# Splitting to train and test sets
x_train, x_test, y_train, y_test = tts(x, real_y)
x_train = x_train.reshape([-1, 28, 28, 1])
x_test = x_test.reshape([-1, 28, 28, 1])

TRAINING_BATCHES = int(len(x_train)/100)
TESTING_BATCHES = int(len(x_test)/35)

x_train_batches = np.split(x_train, TRAINING_BATCHES)
x_test_batches = np.split(x_test, TESTING_BATCHES)
y_train_batches = np.split(y_train, TRAINING_BATCHES)
y_test_batches = np.split(y_test, TESTING_BATCHES)

In [17]:
class DigitRecognizer():
    def __init__(self):
        self.x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
        self.y = tf.placeholder(tf.float32)
        
#         self.W1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
#         self.b1 = tf.Variable(tf.random_normal([32]))
#         self.W2 = tf.Variable(tf.random_normal([14*14*32, 10]))
#         self.b2 = tf.Variable(tf.random_normal([10]))
        
#         conv1 = tf.nn.bias_add(tf.nn.conv2d(self.x, self.W1, strides=[1, 1, 1, 1], padding='SAME'), self.b1)
#         pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        conv1 = tf.layers.conv2d(self.x, filters=32, kernel_size=[3, 3], strides=[1, 1], padding='SAME')
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=[2, 2], padding='SAME')
        relu1 = tf.nn.relu(pool1)
        conv2 = tf.layers.conv2d(relu1, filters=64, kernel_size=[3, 3], strides=[1, 1], padding='SAME')
        pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], strides=[2, 2], padding='SAME')
        
#         logits = tf.nn.bias_add(tf.matmul(tf.reshape(pool1, [-1, 14 * 14 * 32]), self.W2), self.b2)
    
        relu2 = tf.nn.relu(pool2)
        dense1 = tf.layers.dense(tf.layers.flatten(pool2), units=1024)
        relu2 = tf.nn.relu(dense1)
        logits = tf.layers.dense(tf.layers.flatten(relu2), units=10)
        
        self.f = tf.nn.softmax(logits)
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.f, 1), tf.argmax(self.y, 1)), tf.float32))
        self.loss = tf.losses.softmax_cross_entropy(self.y, logits)

In [18]:
model = DigitRecognizer()
mini_op = tf.train.AdamOptimizer(0.001).minimize(model.loss)

session = tf.Session()
session.run(tf.global_variables_initializer())

start = time.time()

EPOCHS = 20

accs = []

for epoch in range(EPOCHS):
    start_epoch = time.time()
    print("Epoch %d" % (epoch + 1))
    for batch in range(TRAINING_BATCHES):
        session.run(mini_op, {model.x: x_train_batches[batch], model.y: y_train_batches[batch]})
    acc = session.run([model.accuracy], {model.x: x_test, model.y: y_test})
    accs.append(acc*100)
    print("Accuracy after %d epoch(s): %.2f%s" % ((epoch+1), acc[0]*100, "%"))
    end_epoch = time.time()
    print("Time for epoch %.d: %.f" % ((epoch+1), (end_epoch - start_epoch)))

end = time.time()
print("Runtime: %.2f" % (end - start))

loss, accuracy = session.run([model.loss, model.accuracy], {model.x: x_test, model.y: y_test})

Epoch 1
Accuracy after 1 epoch(s): 97.28%
Time for epoch 1: 24
Epoch 2
Accuracy after 2 epoch(s): 97.51%
Time for epoch 2: 33
Epoch 3
Accuracy after 3 epoch(s): 98.09%
Time for epoch 3: 20
Epoch 4
Accuracy after 4 epoch(s): 98.13%
Time for epoch 4: 21
Epoch 5
Accuracy after 5 epoch(s): 97.33%
Time for epoch 5: 21
Epoch 6
Accuracy after 6 epoch(s): 97.25%
Time for epoch 6: 21
Epoch 7
Accuracy after 7 epoch(s): 97.56%
Time for epoch 7: 21
Epoch 8
Accuracy after 8 epoch(s): 98.07%
Time for epoch 8: 21
Epoch 9
Accuracy after 9 epoch(s): 98.22%
Time for epoch 9: 21
Epoch 10
Accuracy after 10 epoch(s): 97.78%
Time for epoch 10: 21
Epoch 11
Accuracy after 11 epoch(s): 98.03%
Time for epoch 11: 21
Epoch 12
Accuracy after 12 epoch(s): 97.62%
Time for epoch 12: 21
Epoch 13
Accuracy after 13 epoch(s): 98.38%
Time for epoch 13: 21
Epoch 14
Accuracy after 14 epoch(s): 97.13%
Time for epoch 14: 21
Epoch 15
Accuracy after 15 epoch(s): 98.13%
Time for epoch 15: 21
Epoch 16
Accuracy after 16 epoch(s): 

In [25]:
predictions, accuracy = session.run([model.f, model.accuracy], {model.x: x_test, model.y: y_test})
for p in predictions:
    max = 0
    index = 0
    for n in range(len(p)):
        if p[n] > max:
            max = p[n]
            index = n
    for n in range(len(p)):
        p[n] = 0 if not n == index else 1
        
acc = sum([int(list(predictions[i]) == list(y_test[i])) for i in range(len(predictions))])/len(predictions)*100
print("%.2f%s accuracy" % (acc, '%'))

98.25% accuracy


In [26]:
print(accuracy)

0.98245716


In [ ]:
%matplotlib notebook
plt.plot([x for x in range(1, 21)], accs, label='Accuracies')
plt.legend()
plt.show()